In [41]:
import fiona
import math
import json
import uuid

Calculate summary statistics for private timberland owners

In [37]:
# load sources/private-idustrial-ownership.gpkg 
with fiona.open('sources/private-industrial-ownership.gpkg', 'r') as src:
    private_industrial = [feature for feature in src]
        
    total_industrial_acres = 0
    total_non_industrial_acres = 0
    
    owner_totals = {}
    for feature in private_industrial:
        owner = feature['properties']['Owner']
        
        if feature['properties']['LandManager'] == 'PVI':
            if owner == None: 
                owner = 'All Other'
            if owner not in owner_totals:
                owner_totals[owner] = 0
            owner_totals[owner] += feature['properties']['GISAcres']
            total_industrial_acres += feature['properties']['GISAcres']
        else:
            total_non_industrial_acres += feature['properties']['GISAcres']

    # sort the owner_totals dictionary by total GISAcres
    owner_totals_sorted = sorted(owner_totals.items(), key=lambda x: x[1], reverse=True)
    
    owners = []

    for owner, total in owner_totals_sorted:
        owners.append({'owner': owner, 'acres': round(total), 'percent': round(total / total_industrial_acres * 100)})
        
    out_json = {'owners': owners, 'total_industrial_acres': round(total_industrial_acres), 'total_non_industrial_acres': round(total_non_industrial_acres)}
    
    #save the out_json to a json file
    with open('sources/private-ownership.json', 'w') as f:
        json.dump(out_json, f)


In [43]:
# open sources/ODF-lands.gpkg and sources/oregon-state-land-board.gpkg
with fiona.open('sources/ODF-lands.gpkg', 'r') as src:
    odf_lands = [feature for feature in src]
    for feature in odf_lands:
        feature['properties'] = {'id': uuid.uuid4(), 'manager': 'Oregon Department of Forestry', 'owner_class': 'state'}
    
with fiona.open('sources/oregon-state-land-board.gpkg', 'r') as src:
    oslb_lands = [feature for feature in src]
    for feature in oslb_lands:
        feature['properties'] = {'id': uuid.uuid4(), 'manager': 'Oregon State Land Board', 'owner_class': 'state'}
    
# merge the odf_lands and oslb_lands
merged_lands = odf_lands + oslb_lands

schema = {
    'geometry': 'MultiPolygon',
    'srs': 'EPSG:2992',
    'properties': {
        'id': 'str',
        'manager': 'str',
        'owner_class': 'str'
    }
}

# save the merged_lands to a new file
with fiona.open('sources/merged-lands.gpkg', 'w', driver='GPKG', schema=schema) as dst:
    dst.writerecords(merged_lands)


/var/folders/8h/kk4k5t952ydbc8hfg6nct2940000gn/T/ipykernel_76434/2899456856.py:5: FionaDeprecationWarning: instances of this class -- CRS, geometry, and feature objects -- will become immutable in fiona version 2.0
  feature['properties'] = {'id': uuid.uuid4(), 'manager': 'Oregon Department of Forestry', 'owner_class': 'state'}
/var/folders/8h/kk4k5t952ydbc8hfg6nct2940000gn/T/ipykernel_76434/2899456856.py:10: FionaDeprecationWarning: instances of this class -- CRS, geometry, and feature objects -- will become immutable in fiona version 2.0
  feature['properties'] = {'id': uuid.uuid4(), 'manager': 'Oregon State Land Board', 'owner_class': 'state'}
